In [1]:
from vkprog_data_prep import load_booking_data
import datetime as dt
import pandas as pd

In [7]:
%%time
date_view      = dt.datetime(2018,10,9) # only works for odd calendar weeks!!!

def branchen_data(date_view):
    
    bd = load_booking_data()
    
    kunden_branchen_df  = bd.loc[:,["Endkunde_NR",
                                    "Kampagne_Erfassungsdatum",
                                    "Endkunde_Branchengruppe_ID"] ]

    auftrag_branchen_df = bd.loc[:,["Endkunde_NR",
                                    "Kampagne_Erfassungsdatum",
                                    "Auftrag_Branchengruppe_ID"] ]

    kunden_branchen_df.columns  = ["Endkunde_NR",
                                   "Kampagne_Erfassungsdatum",
                                   "Branchen_ID"]
    
    auftrag_branchen_df.columns = kunden_branchen_df.columns

    branchen_df = (kunden_branchen_df.append(auftrag_branchen_df)
                                     .dropna()
                                     .drop_duplicates() )
    
    # Define filter (boolean pd.Series) based on view_date
    filter_boolean = (bd.loc[:,"Kampagne_Erfassungsdatum"] < date_view)
    
    # Span pivot table with filter:
    branchen_df = (branchen_df
                   .loc[filter_boolean,:]
                   .pivot_table(
                        index      = ["Endkunde_NR"],
                        columns    = ["Branchen_ID"],
                        values     = ["Branchen_ID"],
                        aggfunc    = "count",
                        fill_value = 0) )
    
    #Flatten and rename:
    branchen_df = pd.DataFrame(branchen_df.to_records(index=True))
    new_col_names = [x.replace("('Kampagne_Erfassungsdatum',","B").replace(" '","").replace("')","") for x in branchen_df.columns]
    branchen_df.columns = new_col_names
    
    #Decode into 0/1:
    for branche in branchen_df.columns[1:]:
        branchen_df.loc[:,branche] = branchen_df.loc[:,branche].apply(lambda x: min(x,1))
        
    return branchen_df

Wall time: 0 ns


In [10]:
%%time
damn = branchen_data(date_view=date_view)

2019-10-09 10:39:28 [INFO] Started loading binary file
2019-10-09 10:39:28 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-09 10:39:29 [INFO] Finished loading binary file in 0.16s (0.78s CPU)


Wall time: 13.8 s


In [4]:
a = [1,2,3,4,5]
b = [1,3,5,6]
list(set(a) & set(b))

[1, 3, 5]

In [17]:
list(set(damn.columns) & set(['B105']))

['B105']

In [3]:
10**10


10000000000

In [5]:
import numpy as np


In [8]:
np.inf + 5

inf

In [9]:
import seaborn as sns
sns.load_dataset("flights")
flights = flights.pivot("month", "year", "passengers")
ax = sns.heatmap(flights)

NameError: name 'flights' is not defined